# Análisis de Sentimiento de Google Reviews con SerpAPI y GPT-4o de OpenAI

Este notebook demuestra cómo extraer y analizar opiniones de Google utilizando SerpAPI y luego analizar el sentimiento de esas opiniones utilizando modelos de lenguaje de OpenAI.

- AUTHOR: Albert Gil López
- LINKEDIN: https://www.linkedin.com/in/albertgilopez/
- GITHUB: https://github.com/albertgilopez


In [3]:
# Instalación de librerías necesarias

!pip install pandas
!pip install serpapi
!pip install openai

In [4]:
# Importación de librerías

import pandas as pd
from serpapi import GoogleSearch
import openai

In [ ]:
# Configuración de claves API

SERPAPI_KEY = 'your_serpapi_key_here'  # Reemplazar con tu clave API de SerpAPI
OPENAI_API_KEY = 'your_openai_key_here'  # Reemplazar con tu clave API de OpenAI


## Extracción de Datos de Google Maps
Utilizamos SerpAPI para buscar lugares y extraer opiniones. Asegúrate de utilizar lugares y términos de búsqueda genéricos para evitar exponer datos sensibles.


In [ ]:
# Definición de parámetros para la búsqueda en Google Maps
search_params = {
    "engine": "google_maps",
    "q": "nombre_del_negocio ubicación_general",  # Ejemplo: "Pizzería Barcelona"
    "type": "search",
    "api_key": SERPAPI_KEY
}

search = GoogleSearch(search_params)
results = search.get_dict()
place_id = results['place_id']  # Asumiendo que la búsqueda devuelve un ID de lugar

In [63]:
# Para acceder a las reviews, dentro de la API de Google Maps hay que acceder a Reviews API
# https://serpapi.com/google-maps-reviews-api

# Extracción de opiniones usando el ID del lugar
reviews_params = {
    "engine": "google_maps_reviews",
    "place_id": place_id,
    "api_key": SERPAPI_KEY
}

reviews_search = GoogleSearch(reviews_params)
reviews_results = reviews_search.get_dict()

## Análisis de Sentimientos con OpenAI
Usando los modelos de lenguaje de OpenAI, analizamos el sentimiento de cada opinión obtenida.


In [77]:
# # Utilizando un enfoque de prompting, llamando a la API de OpenAI
# !pip install openai

In [76]:
# Configuración del cliente de OpenAI
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

def analyze_review_sentiment(review_text):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a virtual assistant expert in classifying opinions as positive or negative. Please limit your response to 'NEUTRAL', 'POSITIVO', 'MUY POSITIVO', NEGATIVO', 'MUY NEGATIVO'"},
            {"role": "user", "content": f"{review_text}"}
        ]
    )
    return response.choices[0].message.content

In [ ]:
# Análisis de sentimientos de las opiniones
review_texts = [review['text'] for review in reviews_results['reviews']]
sentiments = [analyze_review_sentiment(text) for text in review_texts]

In [ ]:
# Creación de un DataFrame con los resultados
reviews_df = pd.DataFrame({
    'Review': review_texts,
    'Sentiment': sentiments
})

## Guardar y Visualizar Resultados
Guardamos los resultados en un archivo CSV y visualizamos algunos ejemplos.


In [ ]:
# Guardar los resultados en un archivo CSV
reviews_df.to_csv('reviews_sentiments.csv', index=False)
print(reviews_df.head(10))

Este notebook proporciona una plantilla básica para extraer y analizar opiniones de Google usando SerpAPI y OpenAI.